# Домашнее задание 2. Бейзлайны и детерминированные алгоритмы item-item

[RecSys types](https://drive.google.com/file/d/10i5oN2Cw60oLMvnQ4SKFkkotnhRS7Q8L/view?usp=sharing)

In [441]:
#pip install implicit

In [442]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [443]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [444]:
data['week_no'].nunique()

95

In [445]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [446]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.describe()

,item_id,sales_value
count,8.905100e+04,89051.000000
mean,5.115772e+06,83.458481
std,5.178973e+06,1628.715079
min,2.567100e+04,0.000000
25%,9.665830e+05,3.500000
50%,1.448516e+06,10.780000
75%,9.553042e+06,46.105000
max,1.802456e+07,467993.620000


In [447]:
popularity = data.groupby('item_id')['user_id'].nunique().reset_index()
popularity.describe()

,item_id,user_id
count,8.905100e+04,89051.000000
mean,5.115772e+06,14.759767
std,5.178973e+06,45.904111
min,2.567100e+04,1.000000
25%,9.665830e+05,1.000000
50%,1.448516e+06,2.000000
75%,9.553042e+06,10.000000
max,1.802456e+07,2039.000000


### Train-test split

В рекомендательных системах корректнее использовать train-test split по времени, а не случайно  
Я возьму последние 3 недели в качестве теста

In [448]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [449]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

# 1. Бейзлайны

Создадим датафрейм с покупками юзеров на тестовом датасете (последние 3 недели)

In [450]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [451]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### 1.1 Random recommendation

In [452]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [453]:
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [454]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

CPU times: user 2.99 s, sys: 11.8 ms, total: 3 s
Wall time: 3 s


In [455]:
items

array([ 1004906,  1033142,  1036325, ..., 15722756, 17170636, 15716393])

In [456]:
result.head(2)

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[966035, 1110453, 10254263, 12456588, 929132]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981392, 1194308, 836922, 976047, 17105736]"


### 1.2 Popularity-based recommendation

In [457]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    #print(recs)
    return recs.tolist()

In [458]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 200 ms, sys: 33.8 ms, total: 234 ms
Wall time: 236 ms


In [459]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[966035, 1110453, 10254263, 12456588, 929132]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981392, 1194308, 836922, 976047, 17105736]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### 1.3 Weighted random recommender

- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

*Пример*  
item_1 - 5, item_2 - 7, item_3 - 4  # / sum  
item_1 - 5 / 16, item_2 - 7 / 16, item_3 - 4 / 16

In [460]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [461]:
# считаем вес каждого товара - сумма sales_value
item_sales = data_train.groupby('item_id')['sales_value'].sum()
item_sales

item_id
25671       20.94
26081        0.99
26093        1.59
26190        1.54
26355        1.98
            ...  
17381856     0.00
17382205     7.99
17383227     4.49
17827644     2.50
17829232    11.98
Name: sales_value, Length: 86865, dtype: float64

In [462]:
# для логарифмирования не подходят нули
item_sales.min()

0.0

In [463]:
# заменим нули на 0.00000000001
item_sales.values[item_sales.values==0] = 0.00000000001

In [464]:
log_weights = np.log(item_sales)
log_weights

item_id
25671        3.041661
26081       -0.010050
26093        0.463734
26190        0.431782
26355        0.683097
              ...    
17381856   -25.328436
17382205     2.078191
17383227     1.501853
17827644     0.916291
17829232     2.483239
Name: sales_value, Length: 86865, dtype: float64

In [465]:
#уберем отрицательные
log_weights = log_weights - log_weights.min()

In [466]:
log_weights.sum()

3226462.738864591

In [467]:
log_weights.min()

0.0

In [468]:
log_weights_norm = log_weights/log_weights.sum()

In [469]:
log_weights_norm.sum()

0.9999999999999998

In [470]:
log_weights_norm

item_id
25671       0.000012
26081       0.000011
26093       0.000011
26190       0.000011
26355       0.000011
              ...   
17381856    0.000003
17382205    0.000011
17383227    0.000011
17827644    0.000011
17829232    0.000012
Name: sales_value, Length: 86865, dtype: float64

In [471]:
log_weights_norm.values

array([1.16843191e-05, 1.07384810e-05, 1.08853242e-05, ...,
       1.12070756e-05, 1.10255883e-05, 1.15112433e-05])

In [472]:
%%time

def weighted_random_recommendation(items, weight_list, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    #items = np.array(items)
    #weights_list = np.array(weight_list)
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, p=weight_list, replace=False)
    
    return recs.tolist()

CPU times: user 9 µs, sys: 15 µs, total: 24 µs
Wall time: 27.9 µs


In [473]:
items = log_weights_norm.index
weights = log_weights_norm.values

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, weights, n=5))

In [474]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[966035, 1110453, 10254263, 12456588, 929132]","[6534178, 6533889, 1029743, 6534166, 1082185]","[8020190, 5678054, 9883846, 1071348, 9487424]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[981392, 1194308, 836922, 976047, 17105736]","[6534178, 6533889, 1029743, 6534166, 1082185]","[12385522, 16809017, 987734, 12811505, 8159276]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [475]:
import pickle

In [476]:
with open('predictions_basic1.pickle', 'rb') as handle:
    result = pickle.load(handle)
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13004478, 9527458, 1077643, 986150, 1049733]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[116917, 1029424, 9245375, 9859223, 6602557]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [477]:
items = log_weights_norm.index
weights = log_weights_norm.values

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, weights, n=5))

In [478]:
#pip install ipynb

In [479]:
from ipynb.fs.full.my_metrics import precision_at_k

In [480]:
%%time

metrics = {}

for i in result.columns[1:]:
    metrics[i] = result.apply(lambda row: precision_at_k(row[i], row['actual'],5), axis=1).mean()

ipynb.fs.full.my_metrics:19: RuntimeWarning: invalid value encountered in long_scalars


CPU times: user 1.9 s, sys: 25.7 ms, total: 1.93 s
Wall time: 1.95 s


In [481]:
df = pd.DataFrame(metrics.values(), index = metrics.keys(), columns=['precision_at_k'])
df = df.sort_values(['precision_at_k'])
df

,precision_at_k
random_recommendation,0.000784
weighted_random_recommendation,0.000784
cosine,0.132909
itemitem,0.136925
popular_recommendation,0.155240
tfidf,0.160921
own_purchases,0.201917
actual,1.000000


**плохо показал себя weighted_random_recommendation, получается, что прямо пропорционально полулярности - это еще менее эффективно, чем просто случайный выбор**

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

### 1.1 Random recommendation top 5000

In [482]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [483]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13004478, 9527458, 1077643, 986150, 1049733]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]","[6391167, 9297282, 1014605, 9677861, 1209850]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[116917, 1029424, 9245375, 9859223, 6602557]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]","[10120168, 984140, 953008, 12695733, 157164]"


In [484]:
items = data_train.item_id.unique()
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [485]:
result['random_recommendation_5K'] = result['user_id'].apply(lambda x: random_recommendation(top_5000, n=5))

In [486]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_recommendation_5K
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13004478, 9527458, 1077643, 986150, 1049733]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]","[6391167, 9297282, 1014605, 9677861, 1209850]","[953320, 903674, 1084203, 5591520, 828525]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[116917, 1029424, 9245375, 9859223, 6602557]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]","[10120168, 984140, 953008, 12695733, 157164]","[950982, 828139, 892920, 6463717, 862822]"


### 1.2 Popularity-based recommendation

In [487]:
#здесь нельзя улучшить по топ 5000, так как иы делаем тоже самое, только выбираем не 5000, а 5 самых популярных просто берем

In [488]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    #print(recs)
    return recs.tolist()

### 1.3 Weighted random recommender топ 5000

In [489]:
# считаем вес каждого товара - сумма sales_value
item_sales = data_train.groupby('item_id')['sales_value'].sum()
item_sales_sorted = item_sales.sort_values(axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last', 
                       ignore_index=False, key=None)
item_sales_5000 = item_sales_sorted.head(5000)
item_sales_5000

item_id
6534178     447799.94
6533889      40483.34
1029743      35764.66
6534166      30170.77
1082185      26029.96
              ...    
12188446       265.29
1112387        265.26
981716         265.17
5575861        265.14
915174         265.13
Name: sales_value, Length: 5000, dtype: float64

In [490]:
# нулей нет, отлично
item_sales_5000.min()

265.13

In [491]:
log_weights_5000 = np.log(item_sales_5000)
log_weights_5000

item_id
6534178     13.012102
6533889     10.608646
1029743     10.484716
6534166     10.314629
1082185     10.167003
              ...    
12188446     5.580824
1112387      5.580710
981716       5.580371
5575861      5.580258
915174       5.580220
Name: sales_value, Length: 5000, dtype: float64

In [492]:
log_weights_5000_norm = log_weights_5000/log_weights_5000.sum()

In [493]:
log_weights_5000_norm.sum()

1.0

In [494]:
items_5000 = log_weights_5000_norm.index
weights_5000 = log_weights_5000_norm.values
result['weighted_random_recommendation_5К'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_5000, weights_5000, n=5))



In [495]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_recommendation_5K,weighted_random_recommendation_5К
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13004478, 9527458, 1077643, 986150, 1049733]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]","[6391167, 9297282, 1014605, 9677861, 1209850]","[953320, 903674, 1084203, 5591520, 828525]","[879735, 844165, 984669, 5577022, 850562]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[116917, 1029424, 9245375, 9859223, 6602557]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]","[10120168, 984140, 953008, 12695733, 157164]","[950982, 828139, 892920, 6463717, 862822]","[1032967, 1062340, 1127817, 860703, 1127825]"


In [496]:
%%time

metrics = {}

for i in result.columns[1:]:
    metrics[i] = result.apply(lambda row: precision_at_k(row[i], row['actual'],5), axis=1).mean()

ipynb.fs.full.my_metrics:19: RuntimeWarning: invalid value encountered in long_scalars


CPU times: user 2.41 s, sys: 30.3 ms, total: 2.44 s
Wall time: 2.47 s


In [497]:
df = pd.DataFrame(metrics.values(), index = metrics.keys(), columns=['precision_at_k'])
df = df.sort_values(['precision_at_k'])
df

,precision_at_k
random_recommendation,0.000784
weighted_random_recommendation,0.000784
weighted_random_recommendation_5К,0.005681
random_recommendation_5K,0.005877
cosine,0.132909
itemitem,0.136925
popular_recommendation,0.155240
tfidf,0.160921
own_purchases,0.201917
actual,1.000000


## Улучшим ItemItemRecommender, выбирая число соседей K.

In [498]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [499]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', 
                                  columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [500]:
sparse_user_item

<2499x5001 sparse matrix of type '<class 'numpy.float64'>'
	with 667080 stored elements in Compressed Sparse Row format>

In [501]:
user_item_matrix.shape

(2499, 5001)

In [502]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [503]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [504]:
user_item_matrix

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2498,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [509]:
%%time

for i in range(1, 11):

    model = ItemItemRecommender(K=i, num_threads=3) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)
    
    result['itemitem_' + str(i) + 'k'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=[itemid_to_id[999999]], 
                                        recalculate_user=True)])

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

...

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 18.1 s, sys: 180 ms, total: 18.3 s
Wall time: 18.3 s


In [510]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation,random_recommendation_5K,...,itemitem_1k,itemitem_2k,itemitem_3k,itemitem_4k,itemitem_5k,itemitem_6k,itemitem_7k,itemitem_8k,itemitem_9k,itemitem_10k
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13004478, 9527458, 1077643, 986150, 1049733]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]","[6391167, 9297282, 1014605, 9677861, 1209850]","[953320, 903674, 1084203, 5591520, 828525]",...,"[1082185, 1029743, 995785, 1004906, 1081177]","[1082185, 995242, 1029743, 840361, 904360]","[1082185, 981760, 995242, 1029743, 840361]","[1082185, 981760, 995242, 1127831, 840361]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 995242, 840361]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 995242, 840361, 1127831]","[1082185, 981760, 995242, 840361, 1127831]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[116917, 1029424, 9245375, 9859223, 6602557]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]","[10120168, 984140, 953008, 12695733, 157164]","[950982, 828139, 892920, 6463717, 862822]",...,"[1082185, 1098066, 6534178, 1127831, 1068719]","[1082185, 1098066, 6534178, 826249, 1127831]","[1082185, 981760, 1098066, 6534178, 826249]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]","[1082185, 981760, 1098066, 826249, 995242]"


In [511]:
metrics = {}

for i in result.columns[1:]:
    metrics[i] = result.apply(lambda row: precision_at_k(row[i], row['actual'],5), axis=1).mean()

ipynb.fs.full.my_metrics:19: RuntimeWarning: invalid value encountered in long_scalars


In [512]:
df = pd.DataFrame(metrics.values(), index = metrics.keys(), columns=['precision_at_k'])
df = df.sort_values(['precision_at_k'])
df

,precision_at_k
random_recommendation,0.000784
weighted_random_recommendation,0.000784
weighted_random_recommendation_5К,0.005681
random_recommendation_5K,0.005877
cosine,0.132909
itemitem,0.136925
itemitem_5k,0.151224
popular_recommendation,0.155240
itemitem_6k,0.155436
itemitem_7k,0.160235


**лучший precision у модели с К=2 и 3**